In [1]:
import scanpy as sc
import numpy as np
import functools
import jax
from ot_pert.metrics import compute_metrics, compute_mean_metrics

In [2]:
adata_train_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_300.h5ad"
adata_test_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_300.h5ad"
adata_ood_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_300.h5ad"

In [3]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)


/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [4]:
adata_train.obs["split"].value_counts()

split
train    51882
Name: count, dtype: int64

In [5]:
adata_test.obs["split"].value_counts()

split
test    3100
Name: count, dtype: int64

In [6]:
adata_ood.obs["split"].value_counts()

split
ood    8896
Name: count, dtype: int64

In [7]:
adata_ood.obs["condition"].value_counts()

condition
Givinostat+SRT1720         2260
Cediranib+PCI-34051        2161
Panobinostat+PCI-34051     1814
Panobinostat+Crizotinib    1641
SRT2104+Alvespimycin        520
control                     500
Name: count, dtype: int64

In [8]:
def reconstruct_data(embedding: np.ndarray, projection_matrix: np.ndarray, mean_to_add: np.ndarray) -> np.ndarray:
    return np.matmul(embedding, projection_matrix.T) + mean_to_add

def project_data(data: np.ndarray, projection_matrix: np.ndarray, mean_to_subtract: np.ndarray) -> np.ndarray:
    return np.matmul(data - mean_to_subtract, projection_matrix)

In [9]:
project_data_fn = functools.partial(project_data, projection_matrix = adata_train.varm["PCs"], mean_to_subtract=adata_train.varm["X_train_mean"].T)

In [10]:
OBSM_KEY_DATA_EMBEDDING = "X_pca"

In [11]:
train_data_target_encoded = {}
train_data_target_decoded = {}
train_data_target_encoded_predicted = {}
train_data_target_decoded_predicted = {}
pred_id = adata_train[adata_train.obs["condition"]=="control"].X.A
for cond in adata_train.obs["condition"].cat.categories:
    if cond == "control":
        continue
    train_data_target_encoded[cond] = adata_train[adata_train.obs["condition"]==cond].obsm[OBSM_KEY_DATA_EMBEDDING]
    train_data_target_decoded[cond] = adata_train[adata_train.obs["condition"]==cond].X.A
    train_data_target_decoded_predicted[cond] = pred_id
    train_data_target_encoded_predicted[cond] = project_data_fn(pred_id)

pred_id = adata_test[adata_test.obs["condition"]=="control"].X.A
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"]==cond].obsm[OBSM_KEY_DATA_EMBEDDING]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"]==cond].X.A
    test_data_target_decoded_predicted[cond] = pred_id
    test_data_target_encoded_predicted[cond] = project_data_fn(pred_id)

pred_id = adata_ood[adata_ood.obs["condition"]=="control"].X.A
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"]==cond].obsm[OBSM_KEY_DATA_EMBEDDING]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"]==cond].X.A
    ood_data_target_decoded_predicted[cond] = pred_id
    ood_data_target_encoded_predicted[cond] = project_data_fn(pred_id)



In [12]:
test_metrics_encoded = jax.tree_util.tree_map(compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(compute_metrics, test_data_target_decoded, test_data_target_decoded_predicted)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [13]:
ood_metrics_encoded = jax.tree_util.tree_map(compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")




In [14]:
train_deg_dict = {k: v for k,v in adata_train.uns['rank_genes_groups_cov_all'].items() if k in train_data_target_decoded_predicted.keys()}
test_deg_dict = {k: v for k,v in adata_train.uns['rank_genes_groups_cov_all'].items() if k in test_data_target_decoded_predicted.keys()}
ood_deg_dict = {k: v for k,v in adata_train.uns['rank_genes_groups_cov_all'].items() if k in ood_data_target_decoded_predicted.keys()}

In [15]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]

ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


In [16]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


In [17]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.3804718230438923,
 'deg_ood_sinkhorn_div_1': 32.49083786010742,
 'deg_ood_sinkhorn_div_10': 20.67435073852539,
 'deg_ood_sinkhorn_div_100': 20.047623443603516,
 'deg_ood_e_distance': 39.97579217649731,
 'deg_ood_mmd': 0.0891205241567854}

In [18]:
deg_mean_test_metrics

{'deg_test_r_squared': 0.42197210340034785,
 'deg_test_sinkhorn_div_1': 32.291736144285935,
 'deg_test_sinkhorn_div_10': 19.429015636444092,
 'deg_test_sinkhorn_div_100': 18.320116189809944,
 'deg_test_e_distance': 36.493777684578546,
 'deg_test_mmd': 0.09369762784530328}

In [19]:
mean_test_metrics_encoded

{'encoded_test_r_squared': -0.8121086740963854,
 'encoded_test_sinkhorn_div_1': 121.41888075608473,
 'encoded_test_sinkhorn_div_10': 85.54974688016452,
 'encoded_test_sinkhorn_div_100': 31.1790284376878,
 'encoded_test_e_distance': 60.73159871107918,
 'encoded_test_mmd': 0.06118217906489279}

In [20]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.6786619528456236,
 'decoded_ood_sinkhorn_div_1': 178.64161987304686,
 'decoded_ood_sinkhorn_div_10': 133.90679473876952,
 'decoded_ood_sinkhorn_div_100': 34.363592529296874,
 'decoded_ood_e_distance': 67.16140582960234,
 'decoded_ood_mmd': 0.037720593538678765}

In [21]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/identity"

In [22]:
import os
import pandas as pd
pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, "ood_metrics_encoded.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, "ood_metrics_decoded.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, "test_metrics_encoded.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, "test_metrics_decoded.csv"))
pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, "test_metrics_deg.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, "ood_metrics_deg.csv"))